# **Regression**

## Objectives

*   Fit and evaluate a regression model to predict the sales price, of inherited properties, in Ames, Iowa


## Inputs

* outputs/datasets/collection/HousePricesRecords.csv
* Instructions on which variables to use for data cleaning and feature engineering. These are found in their respective notebooks.

## Outputs

* Train set (features and target)
* Test set (features and target)
* ML pipeline to predict sale price
* Feature Importance Plot
* Performance Plot

---

# Change working directory

Accessing the current directory

In [ ]:
import os
current_dir = os.getcwd()
current_dir

Making sure working in the child of the workspace directory

In [ ]:
os.chdir('/workspaces/milestone-project-heritage-housing-issues')
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

# Load Data

In [ ]:
import numpy as np
import pandas as pd
df = (pd.read_csv("outputs/datasets/collection/HousePricesRecords.csv")
        .drop(labels=['EnclosedPorch', 'WoodDeckSF'], axis=1))
print(df.shape)
df.head()

* Change object type data to numerical data

In [ ]:
df['BsmtExposure'] = df['BsmtExposure'].replace(
    {'None': 0, 'No': 1, 'Mn': 2, 'Av': 3, 'Gd': 4})
df['BsmtFinType1'] = df['BsmtFinType1'].replace(
    {'None': 0, 'Unf': 1, 'LwQ': 2, 'BLQ': 3, 'Rec': 4, 'ALQ': 5, 'GLQ': 6})
# For garage finish, 0 and 1 have been swapped due to getting errors
# further in the project
df['GarageFinish'] = df['GarageFinish'].replace(
    {'None': 1, 'Unf': 0, 'RFn': 2, 'Fin': 3})
df['KitchenQual'] = df['KitchenQual'].replace(
    {'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4})

* They all are so, change all columns of type float, in original dataframe, to integers

In [ ]:
df.info()

---

# MP Pipeline: Regressor

## Create ML pipeline

In [ ]:
from sklearn.pipeline import Pipeline

# Feature Engineering
from feature_engine.imputation import MeanMedianImputer, ArbitraryNumberImputer
from feature_engine.transformation import LogTransformer
from feature_engine.transformation import YeoJohnsonTransformer
from feature_engine.outliers import Winsorizer
from feature_engine.selection import SmartCorrelatedSelection
from feature_engine import transformation as vt

# Feat Scaling
from sklearn.preprocessing import StandardScaler

# Feat Selection
from sklearn.feature_selection import SelectFromModel

# ML algorithms
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor


def PipelineOptimization(model):
    """
    Create a pipeline with the selected data cleaning steps and transformers
    """

    pipeline = Pipeline([

        # data cleaning
        ('median_imputer',  MeanMedianImputer(imputation_method='median',
         variables=['BedroomAbvGr', 'GarageYrBlt', 'MasVnrArea'])),
        ('mean_imputer',  MeanMedianImputer(imputation_method='mean',
         variables=['LotFrontage'])),
        ('arbitrary_number_imputer', ArbitraryNumberImputer(arbitrary_number=0,
         variables=['2ndFlrSF', 'BsmtFinType1', 'GarageFinish'])),

        # feature engineering
        ('log_transformer', vt.LogTransformer(
            variables=['YearBuilt'], base='e')),
        ('yeo_johnson_transformer', vt.YeoJohnsonTransformer(
            variables=['1stFlrSF', 'GarageYrBlt', 'GrLivArea', 'KitchenQual',
                       'OverallQual', 'TotalBsmtSF'])),
        ('winsorizer_iqr', Winsorizer(
            capping_method='iqr', fold=1.5, tail='both',
            variables=['1stFlrSF', 'GarageArea', 'GarageYrBlt',
                       'GrLivArea', 'OverallQual', 'TotalBsmtSF',
                       'YearBuilt'])),
        ('SmartCorrelatedSelection', SmartCorrelatedSelection(
                variables=None, method="spearman", threshold=0.8,
                selection_method="variance")),

        ('feat_scaling', StandardScaler()),

        ('feat_selection',  SelectFromModel(model)),

        ('model', model),
        ])

    return pipeline

## Custom Class for hyperparameter optimisation

* Custom class from the Code Institute Wakthrough Project 02

In [ ]:
from sklearn.model_selection import GridSearchCV


class HyperparameterOptimizationSearch:

    def __init__(self, models, params):
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv, n_jobs, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print(f"\nRunning GridSearchCV for {key} \n")
            model = PipelineOptimization(self.models[key])

            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring)
            gs.fit(X, y)
            self.grid_searches[key] = gs

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                'estimator': key,
                'min_score': min(scores),
                'max_score': max(scores),
                'mean_score': np.mean(scores),
                'std_score': np.std(scores),
            }
            return pd.Series({**params, **d})

        rows = []
        for k in self.grid_searches:
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]
                scores.append(r.reshape(len(params), 1))

            all_scores = np.hstack(scores)
            for p, s in zip(params, all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'min_score',
                   'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns], self.grid_searches

## Split Train and Test Set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['SalePrice'], axis=1),
    df['SalePrice'],
    test_size=0.2,
    random_state=0
)

print("* Train set:", X_train.shape, y_train.shape,
      "\n* Test set:",  X_test.shape, y_test.shape)

## Grid Search CV - Sklearn

### Use default hyperparameters to find most suitable algorithm

In [ ]:
models_quick_search = {
    'LinearRegression': LinearRegression(),
    "DecisionTreeRegressor": DecisionTreeRegressor(random_state=0),
    "RandomForestRegressor": RandomForestRegressor(random_state=0),
    "ExtraTreesRegressor": ExtraTreesRegressor(random_state=0),
    "AdaBoostRegressor": AdaBoostRegressor(random_state=0),
    "GradientBoostingRegressor": GradientBoostingRegressor(random_state=0),
    "XGBRegressor": XGBRegressor(random_state=0),
}

params_quick_search = {
    'LinearRegression': {},
    "DecisionTreeRegressor": {},
    "RandomForestRegressor": {},
    "ExtraTreesRegressor": {},
    "AdaBoostRegressor": {},
    "GradientBoostingRegressor": {},
    "XGBRegressor": {},
}

Do a hyperparameter optimisation search using default hyperparameters

In [ ]:
search = HyperparameterOptimizationSearch(
    models=models_quick_search, params=params_quick_search)
search.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=5)

Check results

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(
    sort_by='mean_score')
grid_search_summary

### Do an extensive search on the most suitable model to find the best hyperparameter configuration.

Define model and parameters, for Extensive Search

In [ ]:
models_search = {
    'ExtraTreesRegressor': ExtraTreesRegressor(random_state=0),
}

params_search = {
    'ExtraTreesRegressor': {
        'model__n_estimators': [50, 100, 150],
        'model__max_depth': [2, None],
        'model__min_samples_split': [1, 2, 3],
        'model__max_features': [2, None],
    },
}

Extensive GridSearch CV

In [ ]:
search = HyperparameterOptimizationSearch(
    models=models_search, params=params_search)
search.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=5)

Check results

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(
    sort_by='mean_score')
grid_search_summary

Parameters for best model

In [ ]:
best_model = grid_search_summary.iloc[0, 0]

In [ ]:
grid_search_pipelines[best_model].best_params_

* These parameters have improved the mean score slightly - an increase to 0.807 from 0.786

Define the best regressor

In [ ]:
best_regressor_pipeline = grid_search_pipelines[best_model].best_estimator_
best_regressor_pipeline

Changing the hyperparameters of the four models resulted in worse scores. The default parameters for LinearRegression results in the best score

## Assess Feature Importance

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

data_cleaning_feat_eng_steps = 7
columns_after_data_cleaning_feat_eng = (
    Pipeline(best_regressor_pipeline.steps[:data_cleaning_feat_eng_steps])
    .transform(X_train)
    .columns
    )

best_features = columns_after_data_cleaning_feat_eng[
    best_regressor_pipeline['feat_selection'].get_support()].to_list()

# Create DataFrame to display feature importance
df_feature_importance = (pd.DataFrame(data={
    'Feature': columns_after_data_cleaning_feat_eng[
        best_regressor_pipeline['feat_selection'].get_support()],
    'Importance': best_regressor_pipeline['model'].feature_importances_})
    .sort_values(by='Importance', ascending=False)
)

# Most important features statement and plot
print(f"* These are the {len(best_features)} most important features in "
      f"descending order. The model was trained on them:\n"
      f"{df_feature_importance['Feature'].to_list()}")

df_feature_importance.plot(
    kind='bar', color='purple', x='Feature', y='Importance'
    )
plt.show()

## Evaluate on Train and Test Sets

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np


def regression_performance(X_train, y_train, X_test, y_test, pipeline):
    print("Model Evaluation \n")
    print("* Train Set")
    regression_evaluation(X_train, y_train, pipeline)
    print("* Test Set")
    regression_evaluation(X_test, y_test, pipeline)


def regression_evaluation(X, y, pipeline):
    prediction = pipeline.predict(X)
    print('R2 Score:', r2_score(y, prediction).round(3))
    print('Mean Absolute Error:', mean_absolute_error(y, prediction).round(3))
    print('Mean Squared Error:', mean_squared_error(y, prediction).round(3))
    print('Root Mean Squared Error:', np.sqrt(
        mean_squared_error(y, prediction)).round(3))
    print("\n")


def regression_evaluation_plots(X_train, y_train, X_test, y_test, pipeline,
                                alpha_scatter=0.5):
    pred_train = pipeline.predict(X_train)
    pred_test = pipeline.predict(X_test)

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
    sns.scatterplot(
        x=y_train, y=pred_train, color='purple',
        alpha=alpha_scatter, ax=axes[0])
    sns.lineplot(x=y_train, y=y_train, color='red', ax=axes[0])
    axes[0].set_xlabel("Actual")
    axes[0].set_ylabel("Predictions")
    axes[0].set_title("Train Set")

    sns.scatterplot(
        x=y_test, y=pred_test, color='purple', alpha=alpha_scatter, ax=axes[1])
    sns.lineplot(x=y_test, y=y_test, color='red', ax=axes[1])
    axes[1].set_xlabel("Actual")
    axes[1].set_ylabel("Predictions")
    axes[1].set_title("Test Set")

    # save plot image to use in the dashboard
    file_path = f'outputs/ml_pipeline/predict_sale_price/v1'
    plt.savefig(f'{file_path}/regression_evaluation_plots.png')

    plt.show()
    plt.close()

Evaluate Performance

In [ ]:
regression_performance(
    X_train, y_train, X_test, y_test, best_regressor_pipeline
    )
regression_evaluation_plots(
    X_train, y_train, X_test, y_test, best_regressor_pipeline
    )

* The R2 on the train set (0.983) and test set (0.845) are both very strong.  
* The data points seem to fit the regression line well although there does seem to be some deviation from this at the higher values
* The R2 scores are above the business expectation of 0.75

---

# Refit pipeline with best features

## Rewrite Pipeline

Rewrite the pipeline including only the 5 most impotant variables('GrLivArea', 'OverallQual', 'KitchenQual', 'TotalBsmtSF', 'GarageArea')

In [ ]:
def PipelineOptimization(model):
    pipeline = Pipeline([

        # Feature engineering
        ('yeo_johnson_transformer', vt.YeoJohnsonTransformer(
            variables=[
                'GrLivArea', 'OverallQual',
                'KitchenQual', 'TotalBsmtSF'
                ])),
        ('winsorizer_iqr',
            Winsorizer(capping_method='iqr', fold=1.5, tail='both',
                       variables=[
                            'GrLivArea', 'OverallQual',
                            'KitchenQual', 'TotalBsmtSF'
                            ])),

        ('feat_scaling', StandardScaler()),

        ('model', model),
        ])

    return pipeline

## Split Train Test Set, only with best features

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['SalePrice'], axis=1),
    df['SalePrice'],
    test_size=0.2,
    random_state=0
)

print("* Train set:", X_train.shape, y_train.shape,
      "\n* Test set:",  X_test.shape, y_test.shape)

Subset Best Features

In [ ]:
X_train = X_train.filter(best_features)
X_test = X_test.filter(best_features)

print("* Train set:", X_train.shape, y_train.shape,
      "\n* Test set:",  X_test.shape, y_test.shape)
X_train.head(3)

## Grid Search CV – Sklearn

We are using the same model from the previous GridCV search

In [ ]:
models_search

And the best parameters from the previous GridCV search

In [ ]:
best_parameters = grid_search_pipelines[best_model].best_params_
best_parameters

params_search = {'ExtraTreesRegressor':  {
    'model__max_features': [2],
    'model__min_samples_split': [3],
    'model__n_estimators[50],
    model__random_state[0]
}
}
params_search

In [ ]:
models_search = {
    'ExtraTreesRegressor': ExtraTreesRegressor(random_state=0),
}

params_search = {
    'ExtraTreesRegressor': {
        'model__n_estimators': [50],
        'model__max_depth': [None],
        'model__min_samples_split': [3],
        'model__max_features': [2],
    },
}

GridSearch CV

In [ ]:
from sklearn.metrics import make_scorer, recall_score
search = HyperparameterOptimizationSearch(
        models=models_search, params=params_search)
search.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=5)

Check results

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(
    sort_by='mean_score')
grid_search_summary

Check the best model

In [ ]:
best_model = grid_search_summary.iloc[0, 0]
best_model

Define the best pipeline

In [ ]:
pipeline = grid_search_pipelines[best_model].best_estimator_
pipeline

* Check R2 score

In [ ]:
regression_performance(X_train, y_train, X_test, y_test, pipeline)
regression_evaluation_plots(X_train, y_train, X_test, y_test, pipeline)

* The train set R2 score is the same as it was before fitting the best model
* The test set R2 score is slightly lower (0.83 vs 0.85) but is still above the business expectation of 0.75

## Assess feature importance

In [ ]:
data_cleaning_feat_eng_steps = 2
columns_after_data_cleaning_feat_eng = (Pipeline(pipeline.steps[
    :data_cleaning_feat_eng_steps])
                                        .transform(X_train)
                                        .columns)

best_features = columns_after_data_cleaning_feat_eng

# Create DataFrame to display feature importance
df_feature_importance = (pd.DataFrame(data={
    'Feature': columns_after_data_cleaning_feat_eng,
    'Importance': pipeline['model'].feature_importances_})
    .sort_values(by='Importance', ascending=False)
)

# Most important features statement and plot
print(f"* These are the {len(best_features)} most important features in  "
      f"descending order. The model was trained on them: "
      f"\n{df_feature_importance['Feature'].to_list()}")

df_feature_importance.plot(
    kind='bar', color='purple', x='Feature', y='Importance')
plt.show()

---

# Push files to the repo

We will generate the following files

* Train set
* Test set
* Modeling pipeline
* features importance plot

In [ ]:
import joblib
import os

version = 'v1'
file_path = f'outputs/ml_pipeline/predict_sale_price/{version}'

try:
    os.makedirs(name=file_path)
except Exception as e:
    print(e)

## Train Set: features and target

In [ ]:
X_train.head()

In [ ]:
X_train.to_csv(f"{file_path}/X_train.csv", index=False)

In [ ]:
y_train

In [ ]:
y_train.to_csv(f"{file_path}/y_train.csv", index=False)

## Test Set: features and target

In [ ]:
X_test.head()

In [ ]:
X_test.to_csv(f"{file_path}/X_test.csv", index=False)

In [ ]:
y_test

In [ ]:
y_test.to_csv(f"{file_path}/y_test.csv", index=False)

## Modelling pipeline

In [ ]:
pipeline

In [ ]:
joblib.dump(value=pipeline, filename=f"{file_path}/pipeline.pkl")

## Feature importance plot

In [ ]:
df_feature_importance.plot(
    kind='bar', color='purple', x='Feature', y='Importance')
plt.show()

In [ ]:
df_feature_importance.plot(kind='bar', color='purple',
                           x='Feature', y='Importance')
plt.savefig(f'{file_path}/features_importance.png', bbox_inches='tight')